In [28]:

# importing required libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
import os, sys, random
import xml.etree.ElementTree as ET
from glob import glob
from shutil import copyfile

In [49]:
def call(data_dir,dir_name):
    txt_file = os.path.join(data_dir,'data.txt')
    with open(txt_file) as fp:
        image_names = fp.read().splitlines()
    
    
    for file in image_names:
        source_file = os.path.join('Dataset','BCCD','JPEGImages',file+'.jpg')
        destination_file = os.path.join(data_dir,'images',file+'.jpg')
        copyfile(source_file, destination_file)
        
        source_file = os.path.join('Dataset','BCCD','Annotations',file+'.xml')
        destination_file = os.path.join(data_dir,'annotations',file+'.xml')
        copyfile(source_file, destination_file)
        
    
    annotations = glob(data_dir+'/annotations/*.xml')
    images = glob(data_dir+'/images/*.jpg')
    
    df = []
    cnt = 0
    for file in annotations:
        filename = file.split("/")[-1]
        #image_file_name = file.split('.')[0]+'.jpg'
        filename =filename.split('.')[0] + '.jpg'
        image_file_name = os.path.join(data_dir,'images',filename)
        row = []
        parsedXML = ET.parse(file)
        for node in parsedXML.getroot().iter('object'):
            blood_cells = node.find('name').text
            xmin = int(node.find('bndbox/xmin').text)
            xmax = int(node.find('bndbox/xmax').text)
            ymin = int(node.find('bndbox/ymin').text)
            ymax = int(node.find('bndbox/ymax').text)

            row = [image_file_name, blood_cells, xmin, xmax, ymin, ymax]
            df.append(row)
            cnt += 1
    
        
    data = pd.DataFrame(df, columns=['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax'])
    csv_file = os.path.join(data_dir,dir_name+'.csv')
    data[['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv(csv_file, index=False)

In [52]:
def create_data():
    train_dir = os.path.join('data','train')
    validation_dir = os.path.join('data','validation')
    test_dir = os.path.join('data','test')
    
    call(train_dir,'train')
    call(test_dir,'test')
    call(validation_dir,'validation')
    

In [53]:
create_data()

In [67]:
def make_annoted_data(train):
    data = pd.DataFrame()
    data['format'] = train['filename']

    # add xmin, ymin, xmax, ymax and class as per the format required
    for i in range(data.shape[0]):
        data['format'][i] = data['format'][i] + ',' + str(train['xmin'][i]) + ',' + str(train['ymin'][i]) + ',' + str(train['xmax'][i]) + ',' + str(train['ymax'][i]) + ',' + train['cell_type'][i]

    data.to_csv('data/train/train.txt', header=None, index=None, sep=' ')

In [68]:
def create_annoted_data():
    train_df = pd.read_csv('data/train/train.csv')
    make_annoted_data(train_df)

In [69]:
create_annoted_data()